In [ ]:
import sympy as sp
import numpy as np
from scipy.optimize import linprog

x0, x1, x2 = sp.symbols('x0 x1 x2')
x_sym = [x0, x1, x2]
x_ = [1, -1, 0] 

f0 = [0.5*x0**2 + x1**2 + 3*x2**2 - 2*x1*x2 + 4, 'max'] 

restricts_sym = [
    1.5*x0**2 + 4*x1**2 + 2.5*x2**2 + 2*x0*x2, 
    2*x0**2 + 3*x1**2 + 0.5*x2**2 + 2*x0*x1,   
    2*x0 + x1 - 2*x2 
]

restrict_val = [7, 3, 2]
z = [[0, 1, 1], [-1, 0, -1], [-1, 1, 1]]  # Возможные направления (для проверки)

def compute_derivates(variables, restrictions, selected_indices=None):
    if selected_indices is None:
        selected_indices = range(len(restrictions))
    return [[sp.diff(restrictions[i], var) for var in variables] for i in selected_indices]

def compute_derivate_in_func(function, variables):
    return [sp.diff(function, var) for var in variables]

def substitute_derivates(derivates):
    if isinstance(derivates[0], list):
        derivates_substituted = []
        for der in derivates:
            substituted_gradient = [expr.subs({x0: x_[0], x1: x_[1], x2: x_[2]}).evalf() for expr in der]
            derivates_substituted.append(substituted_gradient)
        return derivates_substituted
    else:
        return [expr.subs({x0: x_[0], x1: x_[1], x2: x_[2]}).evalf() for expr in derivates]

# Проверка активных ограничений
equals = []
for i, restrict in enumerate(restricts_sym):
    val = restrict.subs({x0: x_[0], x1: x_[1], x2: x_[2]})
    if abs(float(val) - restrict_val[i]) < 1e-6:  # Учет численных погрешностей
        equals.append(i)

if not equals:
    print("Нет активных ограничений. Все направления возможны.")

# Вычисление градиентов активных ограничений
derivates = compute_derivates([x0, x1, x2], restrictions=restricts_sym, selected_indices=equals)
derivates_substituted = substitute_derivates(derivates)

# Градиент целевой функции
func_derivate = compute_derivate_in_func(f0[0], x_sym)
func_substitude_derivate = substitute_derivates(func_derivate)

# --- Новая часть: реализация метода Зойтендейка ---

def formulate_lp_problem(func_gradient, constraint_gradients):
    """
    Формирует задачу линейного программирования для поиска направления z.
    Цель: максимизировать <∇f, z> (для максимизации f).
    Ограничения: <∇g_i, z> <= 0 для активных ограничений (для неравенств) или = 0 (для равенств),
                 ||z||_∞ <= 1 (ограничение на длину шага).
    """
    n = len(func_gradient)
    c = [-float(g) for g in func_gradient]  # Минимизация (-∇f) для максимизации <∇f, z>

    # Ограничения для активных ограничений
    A_ub = []
    b_ub = []
    for i, grad in enumerate(constraint_gradients):
        grad = [float(g) for g in grad]
        if i < len(equals) and equals[i] == 2:  # Если ограничение - равенство (g3)
            A_ub.append(grad)
            b_ub.append(0)
            A_ub.append([-g for g in grad])  # Для равенства добавляем -∇g_i * z <= 0
            b_ub.append(0)
        else:  # Для неравенств
            A_ub.append(grad)
            b_ub.append(0)

    A_bound = np.vstack([np.eye(n), -np.eye(n)])
    b_bound = [1] * n + [1] * n

    A_ub = np.vstack(A_ub + [A_bound]) if A_ub else A_bound
    b_ub = b_ub + b_bound

    return c, A_ub, b_ub

def solve_lp_problem(c, A_ub, b_ub):
    """
    Решает задачу линейного программирования с помощью scipy.optimize.linprog.
    """
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, method='highs')
    if res.success:
        return res.x
    else:
        print("Задача ЛП не имеет решения.")
        return None

def compute_step_size(x_current, z_direction, restrictions, restrict_vals, max_step=1.0, step_reduction=0.5):
    """
    Вычисляет длину шага α, чтобы оставаться в допустимой области.
    Проверяет ограничения g_i(x + αz) <= b_i для неравенств и g_i(x + αz) = b_i для равенств.
    """
    alpha = max_step
    x_new = [x_current[i] + alpha * z_direction[i] for i in range(len(x_current))]

    while alpha > 1e-6:
        feasible = True
        for i, restrict in enumerate(restrictions):
            val = restrict.subs({x0: x_new[0], x1: x_new[1], x2: x_new[2]}).evalf()
            if i == 2:  # Равенство
                if abs(float(val) - restrict_vals[i]) > 1e-6:
                    feasible = False
                    break
            else:  # Неравенство
                if float(val) > restrict_vals[i] + 1e-6:
                    feasible = False
                    break
        if feasible:
            return alpha
        alpha *= step_reduction
        x_new = [x_current[i] + alpha * z_direction[i] for i in range(len(x_current))]
    
    return 0  

def zoutendijk_method(x_initial, f, restrictions, restrict_vals, max_iterations=100, tol=1e-6):
    x_current = x_initial.copy()
    iteration = 0

    while iteration < max_iterations:
        print(f"\nИтерация {iteration + 1}: x = {x_current}")

        # Проверка активных ограничений
        equals = []
        for i, restrict in enumerate(restrictions):
            val = restrict.subs({x0: x_current[0], x1: x_current[1], x2: x_current[2]})
            if abs(float(val) - restrict_vals[i]) < 1e-6:
                equals.append(i)

        # Вычисление градиентов
        derivates = compute_derivates([x0, x1, x2], restrictions, selected_indices=equals)
        derivates_substituted = substitute_derivates(derivates)
        func_derivate = compute_derivate_in_func(f[0], x_sym)
        func_substitude_derivate = substitute_derivates(func_derivate)

        # Формирование и решение задачи ЛП
        c, A_ub, b_ub = formulate_lp_problem(func_substitude_derivate, derivates_substituted)
        z_direction = solve_lp_problem(c, A_ub, b_ub)

        if z_direction is None or np.linalg.norm(z_direction) < tol:
            print("Достигнута сходимость или нет допустимого направления.")
            break

        # Вычисление длины шага
        alpha = compute_step_size(x_current, z_direction, restrictions, restrict_vals)
        print(f"Направление z = {z_direction}, шаг α = {alpha}")

        # Обновление точки
        x_current = [x_current[i] + alpha * z_direction[i] for i in range(len(x_current))]

        # Проверка сходимости по градиенту или шагу
        if alpha < tol:
            print("Сходимость по малому шагу.")
            break

        iteration += 1

    return x_current

# Запуск метода Зойтендейка
x_optimal = zoutendijk_method(x_, f0, restricts_sym, restrict_val)
print(f"\nОптимальная точка: x = {x_optimal}")
print(f"Значение целевой функции: {f0[0].subs({x0: x_optimal[0], x1: x_optimal[1], x2: x_optimal[2]}).evalf()}")



Итерация 1: x = [1, -1, 0]
Направление z = [ 0. -0.  1.], шаг α = 0
Сходимость по малому шагу.

Оптимальная точка: x = [np.float64(1.0), np.float64(-1.0), np.float64(0.0)]
Значение целевой функции: 5.50000000000000
